# Пример сервиса с использованием RAG для вопросно-ответной системы

### Описание шагов с YandexGPT на langchain.com
https://python.langchain.com/v0.2/docs/integrations/text_embedding/yandex/

1. First, you should create service account with the ai.languageModels.user role.
2. Next, you have two authentication options:

*   **IAM token**. You can specify the token in a constructor parameter iam_token or in an environment variable YC_IAM_TOKEN.
*   **API key** You can specify the key in a constructor parameter api_key or in an environment variable YC_API_KEY.

3. To specify the model you can use model_uri parameter, see the documentation for more details.
4. By default, the latest version of text-search-query is used from the folder specified in the parameter folder_id or YC_FOLDER_ID environment variable.

In [1]:
!pip install datasets
!pip install python-dotenv
!pip install langchain
!pip install -U langchain-community
!pip install -U pypdf
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 25.7 MB/

# Пайплайн

![rag_pipeline](rag_pipeline.jpg)

In [2]:
import os
import pandas as pd
from datasets import Dataset
from dotenv import dotenv_values
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings.yandex import YandexGPTEmbeddings
from langchain_community.llms import YandexGPT
from langchain.chains import RetrievalQA

### YC_API_KEY и YC_FOLDER_ID
ключ API и ID папки с проектом Yandex cloud запианы в файл env.txt окружения:
<br><br>
YC_API_KEY=AQVN<...>iKYk<br>
YC_FOLDER_ID=b1g2...<br>

In [5]:
# Загрузим переменные окружения из файла env.txt
env_vars = dotenv_values('env.txt')

# Эксплицитно установим переменные окружения из файла env.txt чтобы перезаписать глобальные переменные, если они есть
for key, value in env_vars.items():
    os.environ[key] = value

# Проверим загрузку переменных окружения
if 'YC_API_KEY' in os.environ:
    print(f"YC_API_KEY и YC_FOLDER_ID успешно загружены")
else:
    print("Ничего не загрузилось")

Ничего не загрузилось


### Загрузка документов:

LangChain предоставляет несколько встроенных загрузчиков документов, которые работают с PDF-файлами, JSON-файлами или Python-файлами в вашей файловой директории.
<br><br>
Для загрузки PDF-файла необходимо установить pypdf <br>


In [18]:
# загрузка документа
loader = PyPDFLoader("Mann 1_+++.pdf")
documents = loader.load()

### Разделение документов на куски (chunking):

Когда документ длинный, необходимо разделить его на части. Существуют различные способы разделения текста. Давайте воспользуемся простейшим методом CharacterTextSplitter, который разделяет текст по символам и измеряет длину фрагмента по количеству символов.

In [19]:
# разбивка документа на chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

### Получение эмбеддингов фрагментов текста:

Чанки текств переводятся в эмбеддинги, чтобы работать с ними как с числовыми данными при семантическом поиске.

In [20]:
# выберем эмбединги какой LLM будем использовать
embeddings = YandexGPTEmbeddings()

### Создание хранилища векторов:

Полученные эмбеддинги сохраним в векторном хранилище, в котором будем искать и извлекать схожие векторы при запросах.

In [23]:
# создадим хранилище текстов с векторами в кач-ве индексов
db = Chroma.from_documents(texts, embeddings)

### Создание интерфейса ретривера:

Хранилище векторов можно представить в интерфейсе ретривера.<br>
Чтобы получить текст, необходимо выбрать тип поиска, например «similarity» для поиска по сходству в ретривере.<br>
Тогда будут выбраны векторы текстовых фрагментов, которые наиболее похожи на вектор вопроса.<br>

*   search_type="similar,
*   search_type="mmr",
*   search_type="similarity",
*   (search_type="similarity_score_threshold",
     search_kwargs={"score_threshold": .5, "k": top_k})

k=2 означает найти 2 наиболее релевантных вектора текстовых фрагментов.

In [24]:
# expose this index in a retriever interface
retriever = db.as_retriever(
    search_type="similarity", search_kwargs={"k": 2}
)

Создание цепочки RetrievalQA для ответов на вопросы:

Цепочка RetrievalQA связывает LLM с интерфейсом ретривера.

Можно выбрать один тип цепочки из четырех вариантов:

* stuff
* map reduce
* refine
* map_rerank
    

1. Тип цепочки по умолчанию = «stuff» включает в запрос ВЕСЬ текст из документов.

2. Тип «map_reduce» разбивает тексты на группы, задает вопрос LLM для каждой группы отдельно и выводит окончательный ответ, основываясь на ответах каждой группы.

3. Тип «refine» разбивает тексты на блоки, представляет первый блок LLM, а затем отправляет ответ вместе со вторым блоком LLM. Он постепенно уточняет ответ, обрабатывая все батчи.

4. Тип «map-rerank» делит тексты на блоки, представляет каждый из них в LLM, возвращает оценку, показывающую, насколько полно он отвечает на вопрос, и определяет окончательный ответ, основываясь на ответах, получивших наибольшее количество баллов из каждого блока.

In [25]:
# создание цепочи ответа на вопрос

qa_chain = RetrievalQA.from_chain_type(
            llm=YandexGPT(temperature=0.0,),
            chain_type="refine",
            retriever=retriever,
            return_source_documents=True,
            verbose=True, # logging
            )

In [27]:
# empty lists
queries = []
rag_answers = []
contexts = []
naive_llm_answers = []
ground_truths = []

# функция, которая заполнит пустые списки выводом longchain
def collect_info(result, query_list, rag_answer_list, context_list):
    query_list.append(result["query"])
    rag_answer_list.append(result["result"])
    current_contexts = []
    for document in result["source_documents"]:
        current_contexts.append(document.page_content)
    context_list.append(current_contexts)

In [28]:
result = qa_chain.invoke("Что такое инфраструктурная власть?")

# store information for evaluation dataset
collect_info(result, queries, rag_answers, contexts)
ground_truths.append("Инфраструктурная власть обозначает способность к реальному проникновению в общество и осуществлению логистических политических решений")

result



> Entering new RetrievalQA chain...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2574 > 1024). Running this sequence through the model will result in indexing errors



> Finished chain.


{'query': 'Что такое инфраструктурная власть?',
 'result': 'Инфраструктурной властью можно назвать способность к реальному проникновению в общество и осуществлению политических решений, объединяя людей для общей деятельности, направленной на достижение целей.',
 'source_documents': [Document(page_content='38участников , вне зависимости  от\xa0размеров  территории  и\xa0коли -\nчества  людей . Основополагающие  структуры  общества  объеди -\nняют  экстенсивную  и\xa0интенсивную  власть  и\xa0таким  образом  спо-\nсобствуют  экстенсивной  и\xa0интенсивной  кооперации  людей  для \nдостижения  их целей  (какими\xa0бы  они ни\xa0были ).\nНо\xa0рассуждения  о\xa0власти  как организации  могут  вызвать  \nложное  представление , согласно  которому  общество  — это всего  \nлишь  совокупность  больших  и\xa0авторитетных  организаций  вла-\nсти. Многие  из\xa0тех , кто также  использует  власть , менее  «орга-\nнизованны », например  рыночный  обмен  включает  коллектив -\nную власть , посколь

In [29]:
# метаданные ответа
for document in result["source_documents"]:
    print(document.metadata)
    print("++++++++++++++++++++++++++")
    print(document.page_content)
    print("==========================")

{'page': 45, 'source': 'Mann 1_+++.pdf'}
++++++++++++++++++++++++++
38участников , вне зависимости  от размеров  территории  и коли -
чества  людей . Основополагающие  структуры  общества  объеди -
няют  экстенсивную  и интенсивную  власть  и таким  образом  спо-
собствуют  экстенсивной  и интенсивной  кооперации  людей  для 
достижения  их целей  (какими бы  они ни были ).
Но рассуждения  о власти  как организации  могут  вызвать  
ложное  представление , согласно  которому  общество  — это всего  
лишь  совокупность  больших  и авторитетных  организаций  вла-
сти. Многие  из тех , кто также  использует  власть , менее  «орга-
низованны », например  рыночный  обмен  включает  коллектив -
ную власть , поскольку  через  обмен  люди  достигают  своих  целей . 
Но  он  также  включает  и  дистрибутивную  власть , поскольку  
лишь  некоторые  обладают  правами  собственности  на  товары  
и услуги . Тем не менее  обменивающиеся  могут  совсем  не иметь  
авторитетной  организации , которая

In [30]:
result = qa_chain.invoke("Кто такой Саргон Аккадский")

# store information for evaluation dataset
collect_info(result, queries, rag_answers, contexts)
ground_truths.append("шумерский царь")
for document in result["source_documents"]:
    print(document.metadata)

result



> Entering new RetrievalQA chain...

> Finished chain.
{'page': 222, 'source': 'Mann 1_+++.pdf'}
{'page': 223, 'source': 'Mann 1_+++.pdf'}


{'query': 'Кто такой Саргон Аккадский',
 'result': 'Саргон Аккадский — это первый известный по имени царь Аккадского царства, объединивший под своей властью шумерские города-государства.',
 'source_documents': [Document(page_content='215анализ  будет  похожего  жанра  в\xa0том  смысле , что он будет  обра-\nщаться  к Саргону  в\xa0качестве  всемирно -исторического  персонажа , \nрепрезентирующего  его эпоху  и\xa0его  династию .\nЗавоевания  Саргона  часто  определяют  как «территориаль -\nную империю ». Я\xa0оспорю  это утверждение  с помощью  доказа -\nтельства  того, что истоки  его власти  лежали  не\xa0столько  в\xa0не-\nпосредственном  контроле  над территорией , сколько  в\xa0личном  \nгосподстве  над подчиненными . Однако  его власть  действитель -\nно простиралась  по\xa0 меньшей  мере на\xa0 несколько  сотен  кило -\nметров  в\xa0длину  и\xa0в\xa0ширину , включая  шумерские  города -госу-\nдарства , северных  областей  Аккада , из\xa0которого  он был родом , \nвплоть  до\xa0Э